In [1]:
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.util import MLUtils
import numpy as np
from pyspark.ml.feature import StandardScaler
import pyspark.sql.functions as f
import pyspark.sql.types
import pandas as pd
from pyspark.sql import Row
from pyspark.ml.feature import VectorAssembler

In [2]:
#creating dataframe in spark
df = pd.read_csv("https://raw.githubusercontent.com/venky14/Machine-Learning-with-Iris-Dataset/master/Iris.csv")
df = df.drop(['Species'], axis = 1)
spark_df = spark.createDataFrame(df)

/databricks/spark/python/pyspark/sql/session.py:577: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 arrow_schema = pa.Schema.from_pandas(pdf, preserve_index=False)

In [3]:
spark_df.show(10)

+---+-------------+------------+-------------+------------+
 Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|
+---+-------------+------------+-------------+------------+
 1| 5.1| 3.5| 1.4| 0.2|
 2| 4.9| 3.0| 1.4| 0.2|
 3| 4.7| 3.2| 1.3| 0.2|
 4| 4.6| 3.1| 1.5| 0.2|
 5| 5.0| 3.6| 1.4| 0.2|
 6| 5.4| 3.9| 1.7| 0.4|
 7| 4.6| 3.4| 1.4| 0.3|
 8| 5.0| 3.4| 1.5| 0.2|
 9| 4.4| 2.9| 1.4| 0.2|
 10| 4.9| 3.1| 1.5| 0.1|
+---+-------------+------------+-------------+------------+
only showing top 10 rows

In [4]:
#single vector column
cols = spark_df.drop('Id').columns
cols

Out[4]: ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']

In [5]:
assembler = VectorAssembler(inputCols=cols, outputCol = 'features')
output_dat = assembler.transform(spark_df).select('Id', 'features')
output_dat.show(5, truncate = False)

+---+-----------------+
Id |features |
+---+-----------------+
1 |[5.1,3.5,1.4,0.2]|
2 |[4.9,3.0,1.4,0.2]|
3 |[4.7,3.2,1.3,0.2]|
4 |[4.6,3.1,1.5,0.2]|
5 |[5.0,3.6,1.4,0.2]|
+---+-----------------+
only showing top 5 rows

In [6]:
#centre the data
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=False, withMean=True)

# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(output_dat)

# Normalize each feature to have unit standard deviation.
scaledData = scalerModel.transform(output_dat)
scaledData.select(['Id', 'scaledFeatures']).show(5, truncate = False) # sample centered data

+---+-----------------------------------------------------------------------------------+
Id |scaledFeatures |
+---+-----------------------------------------------------------------------------------+
1 |[-0.7433333333333341,0.4460000000000002,-2.3586666666666662,-0.9986666666666666] |
2 |[-0.9433333333333334,-0.053999999999999826,-2.3586666666666662,-0.9986666666666666]|
3 |[-1.1433333333333335,0.14600000000000035,-2.458666666666666,-0.9986666666666666] |
4 |[-1.243333333333334,0.04600000000000026,-2.258666666666666,-0.9986666666666666] |
5 |[-0.8433333333333337,0.5460000000000003,-2.3586666666666662,-0.9986666666666666] |
+---+-----------------------------------------------------------------------------------+
only showing top 5 rows

In [7]:
pca = PCA(k=4, inputCol = scaler.getOutputCol(), outputCol="pcaFeatures")

model = pca.fit(scaledData)
transformed_feature = model.transform(scaledData)

In [8]:
np.round(100.00*model.explainedVariance.toArray(),4)

Out[8]: array([92.4616, 5.3016, 1.7185, 0.5183])

In [9]:
pcs = np.round(model.pc.toArray(),4)
pcs

Out[9]: array([[-0.3616, -0.6565, 0.581 , 0.3173],
 [ 0.0823, -0.7297, -0.5964, -0.3241],
 [-0.8566, 0.1758, -0.0725, -0.4797],
 [-0.3588, 0.0747, -0.5491, 0.7511]])

In [10]:
pcs = np.round(model.pc.toArray(),4)
df_pc = pd.DataFrame(pcs, columns = ['PC1','PC2','PC3','PC4'], index = cols)
df_pc

,PC1,PC2,PC3,PC4
SepalLengthCm,-0.3616,-0.6565,0.5810,0.3173
SepalWidthCm,0.0823,-0.7297,-0.5964,-0.3241
PetalLengthCm,-0.8566,0.1758,-0.0725,-0.4797
PetalWidthCm,-0.3588,0.0747,-0.5491,0.7511


In [11]:
transformed_feature.select('pcaFeatures').rdd.map(lambda x: Row(pcaFeatures = str([x for x in list(np.round(x[0].\
                                                    toArray(),4))]))).toDF().show(truncate = False)

+-----------------------------------+
pcaFeatures |
+-----------------------------------+
[2.6842, -0.3266, 0.0215, 0.001] |
[2.7154, 0.1696, 0.2035, 0.0996] |
[2.8898, 0.1373, -0.0247, 0.0193] |
[2.7464, 0.3111, -0.0377, -0.076] |
[2.7286, -0.3339, -0.0962, -0.0631]|
[2.2799, -0.7478, -0.1743, -0.0271]|
[2.8209, 0.0821, -0.2643, -0.0501] |
[2.6265, -0.1704, 0.0158, -0.0463] |
[2.888, 0.5708, -0.0273, -0.0266] |
[2.6738, 0.1067, 0.1915, -0.0559] |
[2.5065, -0.6519, 0.0693, -0.0166] |
[2.6131, -0.0215, -0.1077, -0.1577]|
[2.7874, 0.2277, 0.2003, -0.0072] |
[3.2252, 0.5033, -0.0684, -0.0219] |
[2.6435, -1.1862, 0.1445, 0.157] |
[2.3839, -1.3448, -0.2837, 0.0019] |
[2.6225, -0.8181, -0.1453, 0.1647] |
[2.6483, -0.3191, -0.0334, 0.0761] |
[2.1991, -0.8792, 0.1145, 0.0253] |
[2.5873, -0.5205, -0.2196, -0.0691]|
+-----------------------------------+
only showing top 20 rows

In [12]:
#different scale
df = pd.read_csv('https://courses.edx.org/asset-v1:MITx+15.071x_2a+2T2015+type@asset+block/climate_change.csv')
spark_df = spark.createDataFrame(df)
spark_df.printSchema()

root
-- Year: long (nullable = true)
-- Month: long (nullable = true)
-- MEI: double (nullable = true)
-- CO2: double (nullable = true)
-- CH4: double (nullable = true)
-- N2O: double (nullable = true)
-- CFC-11: double (nullable = true)
-- CFC-12: double (nullable = true)
-- TSI: double (nullable = true)
-- Aerosols: double (nullable = true)
-- Temp: double (nullable = true)

In [13]:
spark_df.show(5)

+----+-----+------------------+------+-------+-------+------------------+------------------+---------+--------+-------------------+
Year|Month| MEI| CO2| CH4| N2O| CFC-11| CFC-12| TSI|Aerosols| Temp|
+----+-----+------------------+------+-------+-------+------------------+------------------+---------+--------+-------------------+
1983| 5| 2.556|345.96|1638.59|303.677| 191.324|350.11300000000006|1366.1024| 0.0863| 0.109|
1983| 6| 2.167|345.52|1633.71|303.746|192.05700000000002| 351.848|1366.1208| 0.0794|0.11800000000000001|
1983| 7|1.7409999999999999|344.15|1633.22|303.795|192.81799999999998| 353.725| 1366.285| 0.0731|0.13699999999999998|
1983| 8| 1.13|342.25|1631.35|303.839| 193.602|355.63300000000004|1366.4202| 0.0673|0.17600000000000002|
1983| 9| 0.428|340.17| 1648.4|303.901| 194.392| 357.465|1366.2335| 0.0619| 0.149|
+----+-----+------------------+------+-------+-------+------------------+------------------+---------+--------+-------------------+
only showing top 5 rows

In [14]:
cols = spark_df.drop('Temp').columns
cols

Out[14]: ['Year',
 'Month',
 'MEI',
 'CO2',
 'CH4',
 'N2O',
 'CFC-11',
 'CFC-12',
 'TSI',
 'Aerosols']

In [15]:
assembler = VectorAssembler(inputCols=cols, outputCol = 'features')
output_dat = assembler.transform(spark_df).select('Temp', 'features')
output_dat.show(5, truncate = False)

+-------------------+-------------------------------------------------------------------------------------------------+
Temp |features |
+-------------------+-------------------------------------------------------------------------------------------------+
0.109 |[1983.0,5.0,2.556,345.96,1638.59,303.677,191.324,350.11300000000006,1366.1024,0.0863] |
0.11800000000000001|[1983.0,6.0,2.167,345.52,1633.71,303.746,192.05700000000002,351.848,1366.1208,0.0794] |
0.13699999999999998|[1983.0,7.0,1.7409999999999999,344.15,1633.22,303.795,192.81799999999998,353.725,1366.285,0.0731]|
0.17600000000000002|[1983.0,8.0,1.13,342.25,1631.35,303.839,193.602,355.63300000000004,1366.4202,0.0673] |
0.149 |[1983.0,9.0,0.428,340.17,1648.4,303.901,194.392,357.465,1366.2335,0.0619] |
+-------------------+-------------------------------------------------------------------------------------------------+
only showing top 5 rows

In [16]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=True)
# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(output_dat)

# Normalize each feature to have unit standard deviation.
scaledData = scalerModel.transform(output_dat)
scaledData.select(['Temp', 'scaledFeatures']).show(5, truncate = False)

+-------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
Temp |scaledFeatures |
+-------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
0.109 |[-1.7057795516992047,-0.4502035379206733,2.4313891926587416,-1.365271034736589,-2.415427998374149,-1.6678691628022821,-2.9977124436890428,-2.549190522748089,0.07917956200445367,2.3973799822837916]|
0.11800000000000001|[-1.7057795516992047,-0.16011422896760338,2.0166409848574762,-1.400061550747296,-2.521395906315111,-1.6546637515752494,-2.961482320340127,-2.5191871845960527,0.12522450770101745,2.159854950192905]|
0.13699999999999998|[-1.7057795516992047,0.1299750799854665,1.562443718730641,-1.5083865665079075,-2.5320361265796754,-1.6452859957763297,-2.9238682359355925,-2.4867282418229264,0.5361255991908349,1.9429842687186174]|
0.17600000000000002|[-1.7057795516992047,0.42006438893853637,0.9110011609759548,-1.658618340190505,-2.5726426814668915,-1.636865153834452,-2.885117326378225,-2.45373321635083,0.8744558523537054,1.7433255460914956] |
0.149 |[-1.7057795516992047,0.7101536978916063,0.16253524355567714,-1.823082597695665,-2.2024064457305257,-1.624999422007254,-2.8460698537375113,-2.4220524581407257,0.407249800093893,1.557436390542106] |
+-------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 5 rows

In [17]:
pca = PCA(k=10, inputCol = scaler.getOutputCol(), outputCol="pcaFeatures")

model = pca.fit(scaledData)
transformed_feature = model.transform(scaledData)

In [18]:
np.round(100.00*model.explainedVariance.toArray(),4)

Out[18]: array([5.19871e+01, 1.46612e+01, 1.08331e+01, 1.01167e+01, 6.23870e+00,
 5.47980e+00, 4.62300e-01, 1.48900e-01, 6.39000e-02, 8.40000e-03])

In [19]:
pcs = np.round(model.pc.toArray(),4)
pcs

Out[19]: array([[-4.237e-01, 1.014e-01, -1.270e-01, -2.120e-02, 2.036e-01,
 9.160e-02, 1.556e-01, -3.187e-01, -4.580e-02, 7.848e-01],
 [ 1.100e-02, 9.100e-03, -6.960e-02, 9.886e-01, 3.190e-02,
 9.060e-02, 5.400e-02, 7.000e-02, -1.810e-02, 1.800e-02],
 [ 6.970e-02, -5.125e-01, -5.834e-01, -8.220e-02, -2.100e-01,
 5.833e-01, -2.860e-02, 1.000e-03, 9.900e-03, 3.000e-04],
 [-4.128e-01, 1.341e-01, -1.264e-01, -9.710e-02, 2.475e-01,
 1.322e-01, 3.007e-01, 7.632e-01, -1.558e-01, -1.017e-01],
 [-4.262e-01, -9.200e-02, -1.070e-02, 4.920e-02, -2.820e-02,
 -8.700e-02, -8.778e-01, 1.638e-01, -7.700e-03, 3.890e-02],
 [-4.170e-01, 1.201e-01, -1.153e-01, 1.400e-02, 2.715e-01,
 1.342e-01, 7.960e-02, -3.456e-01, 6.030e-01, -4.660e-01],
 [-2.862e-01, -4.596e-01, 1.380e-01, 4.660e-02, -4.918e-01,
 -3.956e-01, 2.781e-01, 1.735e-01, 4.000e-01, 1.409e-01],
 [-4.192e-01, -1.951e-01, 9.300e-03, 3.570e-02, -1.428e-01,
 -1.381e-01, 1.653e-01, -3.669e-01, -6.702e-01, -3.664e-01],
 [-5.910e-02, -3.811e-01, 7.480e-01, 3.300e-03, 2.102e-01,
 4.968e-01, 5.900e-03, 5.300e-03, -4.600e-03, 2.660e-02],
 [ 1.687e-01, -5.400e-01, -1.751e-01, 6.500e-03, 6.865e-01,
 -4.215e-01, -1.500e-03, 6.300e-03, -1.660e-02, 5.700e-03]])

In [20]:
pcs = np.round(model.pc.toArray(),4)
df_pc = pd.DataFrame(pcs, columns = ['PC'+str(i) for i in range(1, 11)], index = cols)
df_pc

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10
Year,-0.4237,0.1014,-0.1270,-0.0212,0.2036,0.0916,0.1556,-0.3187,-0.0458,0.7848
Month,0.0110,0.0091,-0.0696,0.9886,0.0319,0.0906,0.0540,0.0700,-0.0181,0.0180
MEI,0.0697,-0.5125,-0.5834,-0.0822,-0.2100,0.5833,-0.0286,0.0010,0.0099,0.0003
CO2,-0.4128,0.1341,-0.1264,-0.0971,0.2475,0.1322,0.3007,0.7632,-0.1558,-0.1017
CH4,-0.4262,-0.0920,-0.0107,0.0492,-0.0282,-0.0870,-0.8778,0.1638,-0.0077,0.0389
N2O,-0.4170,0.1201,-0.1153,0.0140,0.2715,0.1342,0.0796,-0.3456,0.6030,-0.4660
CFC-11,-0.2862,-0.4596,0.1380,0.0466,-0.4918,-0.3956,0.2781,0.1735,0.4000,0.1409
CFC-12,-0.4192,-0.1951,0.0093,0.0357,-0.1428,-0.1381,0.1653,-0.3669,-0.6702,-0.3664
TSI,-0.0591,-0.3811,0.7480,0.0033,0.2102,0.4968,0.0059,0.0053,-0.0046,0.0266
Aerosols,0.1687,-0.5400,-0.1751,0.0065,0.6865,-0.4215,-0.0015,0.0063,-0.0166,0.0057


In [21]:
#Sample of PCs are shown below
transformed_feature.select('pcaFeatures').show(10, truncate = False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
pcaFeatures |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
[5.501970988036302,-1.0339413052597977,-1.5779950804217702,-0.832991661055485,1.905559430787411,1.5937319453830243,-0.0403110689062859,0.08401643092961544,-0.19473812611856275,0.0031577752934648616] |
[5.46453378585945,-0.7238344699645718,-1.270895220090157,-0.5124210499419574,1.8243821396709914,1.4889953863928005,0.08645636889783984,0.04947770224363857,-0.19178963291899545,-0.004471712810368046] |
[5.3961506856879575,-0.5639043994189803,-0.662563858385665,-0.17535283760490525,1.8194226212669404,1.514386582102187,0.11118147154421174,-0.022888284271156564,-0.18393576320209065,0.009963323270308608]|
[5.351135069459376,-0.2881193423387026,0.009440106330344422,0.18045830754316247,1.842019444386322,1.3778204953119992,0.15511720994151904,-0.13228010320614791,-0.17171022589409013,0.026021776908725425] |
[5.179123011662264,0.29773799675881574,0.13014630947257727,0.5633040196980977,1.710962923775042,0.7415552810010654,-0.16777099711697088,-0.19017270348302875,-0.1548563243934938,0.03708572636052898] |
[4.97186688944147,0.7413672699376205,0.07745291896233153,0.9033250401845024,1.5791244883312328,0.3131204297445772,-0.41511567976033076,-0.1198515094256305,-0.16038098430098133,0.030015375973677094] |
[4.912813603109918,0.8820696614467401,0.27851118414289877,1.1925654904710314,1.5558407452054217,0.3605594579771213,-0.24208839320923117,-0.05333241526126373,-0.17609710166217843,0.012599491535093797] |
[4.862838501906204,1.0045987886638743,0.18351805990854098,1.46468115340802,1.4657299929748122,0.39213973382125733,-0.1013316037309332,0.03813627815697401,-0.19399925557866374,-0.010334236110411558] |
[4.735988057456976,1.7327241635072252,-0.6842913749717222,-1.6846181355507768,1.012475110651645,-0.7364241943975094,-0.3100693627421963,-0.16720679041108463,-0.14370037531243832,-0.016593883394477565] |
[4.647266766730868,1.6978402250148643,-0.1058302328988836,-1.383107449304788,1.1171191413111168,-0.5057776373886026,-0.20800689941703648,-0.11576523213983515,-0.15330686838741434,-0.01222696076895241] |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 10 rows